# Домашнее задание №8. Рекуррентные нейронные сети RNN LSTM GRU

## Задание:

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими. Выяснить, какая архитектура больше подходит для нашей задачи:

построить свёрточные архитектуры
построить различные архитектуры с RNN
построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

## Подготовка

In [3]:
pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=c16c86045be893990aba9c678205b178683829c6b703de72be82d4e29a6348fd
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop_words


In [4]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=bc2e0c7f5cfeb9ef7032ddf7cc7b629530213bf3cfb929799b3e9e9dd9704165
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [9]:
pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.1 MB/s eta 0:00:00


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping
from pathlib import Path

In [11]:
# Функция подсчета f1_score
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

## Загрузка данных

In [12]:
# Подключаем диск
from google.colab import drive
drive.mount('/gdrive')
# Устанавливаем путь к папке проекта
ROOT_PATH = '/gdrive/MyDrive/Colab Notebooks/NLP/Захаров Роман/7_Нейронки_для_анализа_текста'

Mounted at /gdrive


In [15]:
# Загружаем данные
df = pd.read_excel(Path(ROOT_PATH).joinpath('отзывы за лето.xls'))
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [16]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [17]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [ ]:
df_train.head()

## Предобработка данных

In [18]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [19]:
# Готовим словарь

text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values
len(text_corpus_train), text_corpus_train # список текстов

(13841,
 array(['наконецтый исправить чушь снеоргинальный прошивка приложение удобно пользоваться',
        'удобно использование', 'отлично', ...,
        'мочь скачать ошибка номер 24', 'сбербанк', 'целое отлично'],
       dtype=object))

In [20]:
# Векторизируем текстовый корпус, превращая каждый текст в последовательность целых чисел (индексы токенов в словаре)
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n', #символы для удаления из текстов
                     lower = False, #перевод к нижнему регистру
                      split = ' ') #разделитель для слов

# Обновляем внутренний словарь на основе списка токенов (используется перед text_to_sequences)
tokenizer.fit_on_texts(text_corpus_train)

# Словарь из токенов
len(tokenizer.index_word), tokenizer.index_word

(10292,
 {1: 'приложение',
  2: 'удобно',
  3: 'работать',
  4: 'удобный',
  5: 'отлично',
  6: 'нравиться',
  7: 'хороший',
  8: 'отличный',
  9: 'телефон',
  10: 'супер',
  11: 'быстро',
  12: 'обновление',
  13: 'пароль',
  14: 'мочь',
  15: 'пользоваться',
  16: 'антивирус',
  17: 'банк',
  18: 'вход',
  19: 'устраивать',
  20: 'сбербанк',
  21: 'раз',
  22: 'прошивка',
  23: 'карта',
  24: 'проблема',
  25: 'рута',
  26: 'программа',
  27: 'ошибка',
  28: 'разработчик',
  29: 'сделать',
  30: 'приходиться',
  31: 'вводить',
  32: 'перевод',
  33: 'счёт',
  34: 'писать',
  35: 'норма',
  36: 'деньга',
  37: 'довольный',
  38: 'около',
  39: 'постоянно',
  40: 'нормально',
  41: 'код',
  42: 'исправить',
  43: 'смс',
  44: 'платёж',
  45: 'понятно',
  46: 'последний',
  47: 'функция',
  48: 'зайти',
  49: 'свой',
  50: 'вылетать',
  51: 'мобильный',
  52: 'стать',
  53: 'шаблон',
  54: 'приходить',
  55: 'возможность',
  56: 'право',
  57: 'делать',
  58: 'иня',
  59: 'проверка',
  

**Преобразуем значения X**

In [21]:
# Преобразует каждый текст из списка в последовательность целых чисел
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

sequences_train[:10] # представление первый 10 текстов в виде последовательности целых чисел

[[610, 42, 775, 3539, 22, 1, 2, 15],
 [2, 181],
 [5],
 [60],
 [2],
 [127, 1],
 [37, 970, 66, 104, 30, 484, 887],
 [102, 51, 1, 690, 139, 732, 106, 1968, 53, 1969, 1625, 2],
 [463, 206, 147, 128, 99, 20, 111, 1059, 51, 9],
 [26, 1371, 888, 5, 3540, 733]]

In [22]:
word_count = len(tokenizer.index_word) + 1  #количество слов в словаре
training_length = max([len(i.split()) for i in text_corpus_train]) #максимальная длина списка токентов из тренировочного датасета
word_count, training_length

(10293, 113)

In [23]:
# Преобразуем списки последовательностей в 2D-массив (количество текстов n_samples, maxlen)
# (короткие последовательности добавляются, длинные -- усекаются. По умолчанию - с начала последовательности)
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)
X_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ...,  27,  84, 223],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]], dtype=int32)

** Преобразуем значения y**
Для подсчета функции потерь y_train и y_test должны быть представлены one-hot кодированием

In [24]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [25]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4])

In [26]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

## Сверточные ахритектуры
CONV модель Keras

In [27]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length)) 
                    # inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    # на вход: (батч, inputlen), на выходе: (батч, inputlen, outputdim)
model.add(Conv1D(128, 3))
                   # 128 -- длина 1D-фильтра, шаг -- 3
                   # на выходе ([128/3], 128)
model.add(GlobalMaxPool1D())
                    # в каждой свертке оставляет максимальный элемент
                    # на выходе ([128/3], 1)
model.add(Dense(10))
               # 10-количество выходов
model.add(Dense(5))
                # num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                # преобразуем вектор в рапределение вероятностей

In [28]:
model.compile(loss='categorical_crossentropy', # y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [29]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 19s 615ms/step - loss: 1.1372 - get_f1: 0.4072 - val_loss: 0.8624 - val_get_f1: 0.7071
Epoch 2/20
25/25 [==============================] - 16s 654ms/step - loss: 0.7602 - get_f1: 0.7467 - val_loss: 0.7421 - val_get_f1: 0.7415


In [53]:
score_conv = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_conv[0])
print('Test f1_score:', score_conv[1])

14/14 [==============================] - 7s 484ms/step - loss: 0.9554 - get_f1: 0.7151


Test loss: 0.9554319381713867
Test f1_score: 0.7151089310646057


## Реккурентные сети
Обучим рекуррентные нейронные сети SimpleRNN, LSTM и GRU на одних и тех же параментрах сети и сравним показатели метрики f1_score.

### SimpleRNN

In [30]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [32]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 14s 469ms/step - loss: 1.1314 - get_f1: 0.4793 - val_loss: 0.8569 - val_get_f1: 0.7262
Epoch 2/20
25/25 [==============================] - 9s 359ms/step - loss: 0.8584 - get_f1: 0.7204 - val_loss: 0.7886 - val_get_f1: 0.7353
CPU times: user 25.3 s, sys: 924 ms, total: 26.2 s
Wall time: 22.8 s


In [33]:
score_simplernn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_simplernn[0])
print('Test f1_score:', score_simplernn[1])

14/14 [==============================] - 1s 89ms/step - loss: 0.7795 - get_f1: 0.7438


Test loss: 0.7795130014419556
Test f1_score: 0.7438198328018188


### LSTM

In [34]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64, recurrent_dropout=0.2))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [35]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [36]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 41s 1s/step - loss: 1.3604 - get_f1: 0.2451 - val_loss: 0.9648 - val_get_f1: 0.7008
Epoch 2/20
25/25 [==============================] - 39s 2s/step - loss: 0.9726 - get_f1: 0.6700 - val_loss: 0.8438 - val_get_f1: 0.7319
CPU times: user 1min 33s, sys: 3.42 s, total: 1min 36s
Wall time: 1min 20s


In [37]:
score_lstm= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_lstm[0])
print('Test f1_score:', score_lstm[1])

14/14 [==============================] - 7s 500ms/step - loss: 0.8276 - get_f1: 0.7413


Test loss: 0.8276181817054749
Test f1_score: 0.7412915825843811


### GRU

In [38]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64,recurrent_dropout=0.2))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [39]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [40]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 40s 1s/step - loss: 1.3969 - get_f1: 0.1094 - val_loss: 0.9430 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 32s 1s/step - loss: 0.9082 - get_f1: 0.7156 - val_loss: 0.8057 - val_get_f1: 0.7359
CPU times: user 1min 15s, sys: 2.56 s, total: 1min 17s
Wall time: 1min 12s


In [41]:
score_gru = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_gru[0])
print('Test f1_score:', score_gru[1])

14/14 [==============================] - 2s 176ms/step - loss: 0.7838 - get_f1: 0.7459


Test loss: 0.7837982773780823
Test f1_score: 0.7458855509757996


In [42]:
# Сравним показатели метрик всех трёх реккурентных моделей

result_RNN = pd.DataFrame({'model': ['SimpleRNN', 'LSTM', 'GRU'], 'f1_score': [score_simplernn[1], score_lstm[1], score_gru[1]]})
result_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
2,GRU,0.745886
0,SimpleRNN,0.743820
1,LSTM,0.741292


### Вывод:
SimpleRNN обучалась 22.8 s, LSTM -- 1min 20s, GRU -- 1min 12s.
Лучшие показатели метрики f1_score у модели GRU, время обучения - среднне. Значительно быстрее всех работает SimpleRNN, при этом показатели метрики у неё отличаются на 0.002 от GRU.
При отсутствии требования наилучшей метрики сетью выбора является SimpleRNN!

## Совместные архитектуры

Рассмотрим совместные архитектуры CNN + RNN и оценим показатели метрики таких моделей. В качестве RNN модели возьмем архитектуру LSTM.



### CNN --> RNN

In [43]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [44]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [45]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 50s 2s/step - loss: 1.1806 - get_f1: 0.4105 - val_loss: 0.8803 - val_get_f1: 0.7304
Epoch 2/20
25/25 [==============================] - 40s 2s/step - loss: 0.8735 - get_f1: 0.7065 - val_loss: 0.7746 - val_get_f1: 0.7367
CPU times: user 1min 46s, sys: 5.31 s, total: 1min 51s
Wall time: 1min 30s


In [46]:
score_cnnrnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_cnnrnn[0])
print('Test f1_score:', score_cnnrnn[1])

14/14 [==============================] - 4s 292ms/step - loss: 0.7658 - get_f1: 0.7432


Test loss: 0.7658165693283081
Test f1_score: 0.7432032227516174


## RNN --> CNN

In [47]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
  #return_sequences -- слоит ли возвращать последний вывод выходной последовательности или целую последовательность
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
#model.add(Activation("relu"))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [48]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [49]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 45s 1s/step - loss: 1.2593 - get_f1: 0.3501 - val_loss: 1.0366 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 34s 1s/step - loss: 1.0772 - get_f1: 0.6137 - val_loss: 0.9788 - val_get_f1: 0.7005
CPU times: user 1min 46s, sys: 3.36 s, total: 1min 49s
Wall time: 1min 19s


In [50]:
score_rnncnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_rnncnn[0])
print('Test f1_score:', score_rnncnn[1])

14/14 [==============================] - 6s 439ms/step - loss: 0.9554 - get_f1: 0.7151


Test loss: 0.9554319381713867
Test f1_score: 0.7151089310646057


In [52]:
result_CNN_RNN = pd.DataFrame({'model': ['CNN_RNN', 'RNN_CNN'], 'f1_score': [score_cnnrnn[1], score_rnncnn[1]]})
result_CNN_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
0,CNN_RNN,0.743203
1,RNN_CNN,0.715109


### Вывод:

Модель CNN_RNN обучалась 1min 30s, модель RNN_CNN -- 1min 19s.

Лучшие показатели метрики f1_score у модели CNN + LSTM, но обучается она дольше, чем LSTM + CNN.

## Выводы:

Оценим сводную таблицу метрик всех рассмотренных в задании моделей.

In [54]:
result_CNN = pd.DataFrame({'model': 'CNN', 'f1_score': [score_conv[1]]})
result_models = pd.concat([result_CNN, result_RNN, result_CNN_RNN], ignore_index=True)
result_models.sort_values(by='f1_score', ascending=False)

,model,f1_score
3,GRU,0.745886
1,SimpleRNN,0.743820
4,CNN_RNN,0.743203
2,LSTM,0.741292
0,CNN,0.715109
5,RNN_CNN,0.715109


**Лучшие показатели метрики f1_score у модели GRU.**